# RSA 原理與實作

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import binascii
import random

### 質因數分解

In [2]:
def factorize(x):
    is_prime = True
    for i in sp.primerange(0, x ** 0.5 + 1):
        if x % i == 0:
            j = x // i
            is_prime = False
            return [i, j]
    if is_prime:
        return [0, x]
    
print(factorize(2 ** 10))

[2, 512]


In [3]:
def factorize2(x):
    fact_list = list()
    if sp.isprime(x) == False:
        fact = 1
        while fact != 0:
            fact, x = factorize(x)    
            if fact != 0:
                fact_list.append(fact)
        if x != 1:
            fact_list.append(x)
    else:
        fact_list.append(x)
    return fact_list

print(factorize2(2 ** 10))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### 歐幾里德演算法（輾轉相除法）

$
\forall m , n, q, r \in Z \mbox{ , if } m = nq + r \mbox{ , then } gcd(m,n) = gcd(n, r) \\
$

$ 
\begin{cases}
gcd(n, 0) & \mbox{, if $n > 0$} \\ 
gcd(m, n) = gcd(m, r) & \mbox{, if $m > n$} \\ 
\end{cases}
$

$ 
\mbox{證明： } gcd(m,n) = gcd(n, r) \\ 
\mbox{令： } gcd(m,n) = g1, gcd(n, r) = g2 \\
$

$
\because g1 | m \mbox{ and } g1 | n \\
\therefore g1 | (m - nq) \\
\rightarrow g1 | r \\
\because g2 \mbox{ is the greatest common divisor of } n, r\\
\therefore g2 \geq g1 \\
$
$
\because g2 | n \mbox{ and } g2 | r \\
\therefore g2 | (nq + r) \\
\rightarrow g2 | m \\
\because g1 \mbox{ is the greatest common divisor of } m, n\\
\therefore g1 \geq g2 \\
$

$
\because g2 \geq g1 \mbox{ and } g1 \leq g2 \\
\therefore g1 = g2 \\
\rightarrow gcd(m,n) = gcd(q,r) \\
$

In [4]:
def greatest_common_divisor(a, b):
    if b == 0:
        return a
    else:
        return greatest_common_divisor(b, a % b)

### 歐拉函數

定義：$ \phi(n): $ { $ 1, 2 ... n - 1 $ } 中與 n 互質的元素個素 

$
\forall n \in Z, n = p{_1}^{e_1} p{_2}^{e_2} p{_3}^{e_3}... p{_k}^{e_k} \mbox{ ($\forall i \in Z, i\leq k, p_i$ is prime)} \\
$

$
\phi(n) = n\prod^{k}_{i = 1}(1 - \frac{1}{p_i})
$

證明：$ \phi(n) = n-1 \iff n $ is prime

$
\Rightarrow \quad\\
\textrm{prove by contradiction, suppose n is not prime} \\
\exists m \in Z^+, 1 < m < n \textrm{ st } m | n \\
\rightarrow \phi(n) < n - 1 \textrm{ 矛盾} \\
\rightarrow \textrm{ n is prime} \\
$

$ 
\Leftarrow \quad \\
\because gcd(m, n) = 1, \forall m = 1, 2, ... n - 1 \\
\therefore \phi(n) = n -1 \\
$

In [5]:
def euler_function(p, q):
    return (p - 1) * (q - 1)

# RSA 實作

1. 亂數生成兩相異質數 $ p, q $ 

2. 根據歐拉函數， $ \phi(n) = \phi(p) * \phi(q) $

### 轉碼

$ \mbox{UTF-8} \rightarrow Decimal $

In [6]:
def string_to_int(string):
    byte = string.encode()               # string to byte
    hexadecimal = binascii.hexlify(byte) # byte to hex
    decimal = int(hexadecimal, 16)       # hex to dec
    return decimal

$ Decimal \rightarrow \mbox{UTF-8} $

In [7]:
def int_to_string(int):
    hexadecimal = hex(int)               # dec to hex
    byte = hexadecimal[2:].encode()      # hex to byte (type)
    byte = binascii.unhexlify(byte)      # hex to byte (coding)
    string = byte.decode('utf-8')        # byte to utf-8
    return string

In [8]:
string = 'Hello World! 你好！'
integer = string_to_int(string)
print(integer)
message = int_to_string(integer)
print(message)

27086628833817823194713031788698839469377765346622593
Hello World! 你好！


### 加密函數

In [9]:
def encode(plaintext, public_key, n):
    cyphertext = (plaintext ** public_key) % n    
    return cyphertext

### 解密函數

In [10]:
def decode(cyphertext, private_key, n):
    plaintext = (cyphertext ** private_key) % n
    return plaintext

In [11]:
def decode2(cyphertext, private_key_fact_list, n):
    for i in private_key_fact_list:
        cyphertext = (cyphertext ** i) % n
    plaintext = cyphertext
    return plaintext

### 產生鑰匙

In [12]:
def generate_key():
    public_key, private_key = 0, 0
    prime = list(sp.primerange((10 ** 3), (10 ** 4)))
    while (public_key == 0) or (private_key == 0):
        p, q = random.choices(prime, k=2)
        n = p * q
        o = euler_function(p, q)
        public_key, private_key = factorize(o * 10 + 1)
        gcd = greatest_common_divisor(public_key, o)
        remainder = public_key * private_key % o
        print('gcd: %i' % gcd)
        print('remainder: %i' % remainder)
        if gcd == 1 and remainder == 1:
            print('generate key succes\n')
            break
        else:
            print('generate key fail\n')
    return p, q, public_key, private_key, n

In [13]:
p, q, public_key, private_key, n = generate_key()
private_key_fact_list = factorize2(private_key) # 分解 private key，避免解密時間過長
print('p: %i' % p)
print('q: %i' % q)
print('public key: %i' % public_key)
print('private key: %i = %s' % (private_key, str(private_key_fact_list)))
print('n: %i' % n)

gcd: 1
remainder: 1
generate key succes

p: 6151
q: 1217
public key: 199
private key: 375799 = [375799]
n: 7485767


### 加密

In [14]:
plaintext = input()
plaintext_list = list()
cyphertext_list = list()
for i in plaintext:
    integer = string_to_int(i)
    plaintext_list.append(integer)
    cyphertext = encode(integer, public_key, n)
    cyphertext_list.append(cyphertext)
print('plain text: %s' % str(plaintext_list))
print('cypher text: %s' % str(cyphertext_list))

Hello World!
plain text: [72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33]
cypher text: [4468527, 3570057, 6474546, 6474546, 175601, 3120260, 6332499, 175601, 4265094, 6474546, 6874139, 4444342]


### 解密

In [15]:
message = ''
for i in cyphertext_list:
    plaintext = decode2(i, private_key_fact_list, n)
    message += int_to_string(plaintext)
print(message)

Hello World!
